<a href="https://colab.research.google.com/github/minsuk-heo/tf2/blob/master/jupyter_notebooks/10.Word2Vec_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentence Classification using LSTM and Pretrained Word2Vec

In [33]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

In [34]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

import tensorflow_hub as hub
import numpy as np

In [35]:
# Load Pretrained Word2Vec
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [36]:
def get_max_length(df):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in df['review']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def get_word2vec_enc(reviews):
    """
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    encoded_reviews = []
    for review in reviews:
        tokens = review.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_reviews.append(word2vec_embedding)
    return encoded_reviews
        
def get_padded_encoded_reviews(encoded_reviews):
    """
    for short sentences, we prepend zero padding so all input to RNN has same length
    """
    padded_reviews_encoding = []
    for enc_review in encoded_reviews:
        zero_padding_cnt = max_length - enc_review.shape[0]
        pad = np.zeros((1, 250))
        for i in range(zero_padding_cnt):
            enc_review = np.concatenate((pad, enc_review), axis=0)
        padded_reviews_encoding.append(enc_review)
    return padded_reviews_encoding

def answer_encode(answer):
    """
    return one hot encoding for Y value
    """
    if answer == 'positive':
        return [1,0]
    else:
        return [0,1]
    
def preprocess(df):
    """
    encode text value to numeric value
    """
    # encode words into word2vec
    reviews = df['review'].tolist()
    
    encoded_reviews = get_word2vec_enc(reviews)
    padded_encoded_reviews = get_padded_encoded_reviews(encoded_reviews)
    # encoded answer
    answers = df['answer'].tolist()
    encoded_answer = [answer_encode(answer) for answer in answers]
    X = np.array(padded_encoded_reviews)
    Y = np.array(encoded_answer)
    return X, Y 

# Preprocess (encode text to number)

In [68]:
movie_reviews_train = [
         {'review': 'Andy ate an apple',                          'answer': 'positive'},
         {'review': 'Jack was shocked by his test score',         'answer': 'negative'},
         {'review': 'David hung out with his friends',            'answer': 'negative'},
         {'review': 'Professor Kim gave students a present.',     'answer': 'positive'}
    ]
df = pd.DataFrame(movie_reviews_train)

# max_length is used for max sequence of input
max_length = get_max_length(df)

train_X, train_Y = preprocess(df)

# Build Model

In [69]:
# LSTM model
model = Sequential()
model.add(LSTM(32))
model.add(Dense(2, activation='softmax'))

In [70]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train

In [71]:
print('Train...')
model.fit(train_X, train_Y,epochs=50)

Train...
Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 0.7065 - accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 10ms/step - loss: 0.6946 - accuracy: 0.5000
Epoch 3/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6833 - accuracy: 0.7500
Epoch 4/50
1/1 [==============================] - 0s 11ms/step - loss: 0.6724 - accuracy: 0.7500
Epoch 5/50
1/1 [==============================] - 0s 11ms/step - loss: 0.6617 - accuracy: 0.7500
Epoch 6/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6511 - accuracy: 0.7500
Epoch 7/50
1/1 [==============================] - 0s 7ms/step - loss: 0.6404 - accuracy: 0.7500
Epoch 8/50
1/1 [==============================] - 0s 6ms/step - loss: 0.6297 - accuracy: 0.7500
Epoch 9/50
1/1 [==============================] - 0s 8ms/step - loss: 0.6188 - accuracy: 0.7500
Epoch 10/50
1/1 [==============================] - 0s 13ms/step - loss: 0.6078 - accuracy: 0.7500
Epoch 11/50
1/1 [==========

In [72]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 32)                36224     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 36,290
Trainable params: 36,290
Non-trainable params: 0
_________________________________________________________________


# Test
your model can predict correctly even for unseen words from training.  
This is the most benefit of using pretrained word embedding.  
Why? pretrained embedding will encode [better], [nice] to similar vector of [best]  
even if these words were not in train.  
therefore, the input vector to RNN is similar, so correct answers for even these unseen words.

In [79]:
"""
movie_reviews_train = [
         {'review': 'Andy ate an apple',                          'answer': 'positive'},
         {'review': 'Jack was shocked by his test score',         'answer': 'negative'},
         {'review': 'David hung out with his friends',            'answer': 'negative'},
         {'review': 'Professor Kim gave students a present.',     'answer': 'positive'}
    ]
"""
movie_reviews_test = [
         {'review': 'Andy did not ate an apple',                  'answer': 'negative'},
         {'review': 'Jack was happy with his test score',         'answer': 'positive'},
         {'review': 'David hang out with his friends',            'answer': 'negative'},
         {'review': 'Professor Kim gave students a present',      'answer': 'positive'},
         {'review': 'Professor gave students a reward',           'answer': 'positive'}
    ]
test_df = pd.DataFrame(movie_reviews_test)
# print(test_df)

test_X, test_Y = preprocess(test_df)
# print(test_X)
# print(test_Y)

score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

1/1 - 0s - loss: 1.1236 - accuracy: 0.6000
Test score: 1.1236151456832886
Test accuracy: 0.6000000238418579
